# Web Scraping - Rate Prediction

In [1]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

# Importing selenium webdriver 
from selenium import webdriver

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

#Importing requests
import requests

# importing regex
import re

import warnings
warnings.filterwarnings('ignore')

In [2]:
driver = webdriver.Chrome(r'C:\chromedriver_win32 (2)\chromedriver')

In [3]:
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(2)
#clicking on cancel buttopn 
driver.find_element_by_xpath("//div[@class='_2QfC02']/button").click()

In [4]:
srch_items = ['mobile','smartwatch','laptop','monitor','printer','professional camera','home theater','router','headphone']
title = []
review_text = []
ratings = []

In [5]:
def scrap():    
        for i in driver.find_elements_by_xpath("//div[@class='t-ZTKy']"):
            review_text.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']"):
            title.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']/../div"):
            ratings.append(i.text)
        return

In [6]:
urls=[]
for i in srch_items:
    #Find search bar
    srchBar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input")
    srchBar.clear()
    srchBar.send_keys(i)
    #clicking on search button
    driver.find_element_by_xpath("//button[@class='L0Z3Pu']").click()
    time.sleep(3)
    page = []
    for i in driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a"):
        page.append(i.get_attribute('href'))
    for i in page[0:4]:
        driver.get(i)
        time.sleep(3)
        items = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
        for i in items:
            urls.append(i.get_attribute('href'))

In [7]:
len(urls)

480

In [8]:
for i in urls:
    driver.get(i)
    time.sleep(3)
    for _ in range(2):
        driver.execute_script("window.scrollBy(0,6000)")
        time.sleep(1)
        #clicking on all reviews
    try:
        btn=driver.find_element_by_xpath("//div[@class='_2c2kV-']/following::a")
        lnk = btn.get_attribute('href')
        driver.get(lnk)
        time.sleep(2)
    except NoSuchElementException:
        pass

    scrap()        
    try:
        n_page=driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a")
        np=[]
        for i in n_page:
            np.append(i.get_attribute('href'))
        for i in np[0:18]:
            driver.get(i)
            time.sleep(2)
            scrap()
    except: continue

In [9]:
len(ratings), len(review_text), len(title)

(31480, 31480, 31480)

In [11]:
#### Creating a dataframe
data = list(zip(title,review_text,ratings))
df1 = pd.DataFrame(data, columns = ["review_title","review_text","ratings"])
df1

,review_title,review_text,ratings
0,Worth every penny,Best part is super battery. More than Wow 💖. S...,5
1,Super!,Best mobile,5
2,Nice,Its sound is not clear,3
3,Good quality product,Good phone,4
4,Perfect product!,Best phone in this price.. Amazing!,5
...,...,...,...
31475,Highly recommended,Fantastic product and great service by flipkar...,5
31476,Worst experience ever!,Product got damaged within two days after retu...,1
31477,Good,Good camera from Canon dslr I was using for on...,4
31478,Pretty good,Very good.. Product but ISO performance not th...,4


In [12]:
df1.to_excel('Rate_Prediction.xlsx')